In [12]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

df = pd.read_csv("financial_transactions_resized.csv")

X = df.drop('isFraud', axis=1)
y = df['isFraud']


In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [14]:
scaler = StandardScaler()
x_train, x_test = scaler.fit_transform(x_train), scaler.transform(x_test)
x_train, x_test = torch.tensor(x_train, dtype=torch.float32), torch.tensor(x_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train.values, dtype=torch.long), torch.tensor(y_test.values, dtype=torch.long)

ValueError: could not convert string to float: 'PAYMENT'

In [ ]:
train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=32)

model = nn.Sequential(
    nn.Linear(x_train.shape[1], 128), nn.ReLU(),
    nn.Linear(128, 64), nn.ReLU(),
    nn.Linear(64, 32), nn.ReLU(),
    nn.Linear(32, 3)
)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(100):
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(batch_x), batch_y)
        loss.backward()
        optimizer.step()

with torch.no_grad():
    y_pred = model(x_test).argmax(dim=1)
    accuracy_score = accuracy_score(y_test,y_pred)
    f1_score = f1_score(y_test, y_pred, average="weighted")
    recall_score = recall_score(y_test, y_pred, average="weighted")
    precision_score = precision_score(y_test, y_pred, average = "weighted")
    print(f"Accuracy Score:{accuracy_score*100:.2f}%")
    print(f'Average F1-score: {f1_score * 100:.2f}%')
    print(f"Recall Score: {recall_score*100:.2f}%")
    print(f"Precision Score: {precision_score*100:.2f}%")

